
# Youtube - Scaper



In [ ]:

import requests
from bs4 import BeautifulSoup


headers= {"User-Agent": "Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)"}
def video_finder():
    word = input("Enter video title: ")

    params = {
        'search_query': word
    }

    vid = requests.get('https://www.youtube.com/results', params=params, headers=headers)
    soup = BeautifulSoup(vid.content, features='html.parser')
    a_Tag = soup.find('a', class_="yt-uix-tile-link yt-ui-ellipsis yt-ui-ellipsis-2 yt-uix-sessionlink spf-link", href=lambda h: h.startswith('/watch?'))
    Video_name = a_Tag.text
    Video_id = a_Tag['href']
    video_link = 'https://www.youtube.com' + Video_id
    print(Video_name)
    print(video_link)

video_finder()


In [ ]:

# Scraping transaction : 

def _scrape_and_store(tx, httpUrl): # "tx" is a neo4j transaction...
    #... TODO ...
    result = tx.run("")
    return result.single()[0] # TODO


## Using Neo4j for data storage : ##

In [ ]:
from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'


In [ ]:
httpUrls = [
    #...
]


driver = GraphDatabase.driver(uri, auth=(user, password))

with driver.session() as session:
    for url in httpUrls :
        result = session.write_transaction(_scrape_and_store, url)
        print(result)

driver.close()